# Cálculo Vencimento Gestores

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from utils_ccd import get_connection, get_informacoes_processo

from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate

from pydantic import BaseModel, Field

conn = get_connection()

In [2]:
sql_gestores = """
SELECT CONCAT(p.numero_processo, '/', p.ano_processo) as processo, p.codigo_tipo_processo, p.assunto, 
r.nome, pm.Descricao as marcador, p.IdProcesso as id_processo
FROM processo.dbo.Processos p 
	INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON p.IdProcesso = pmp.IdProcesso
	INNER JOIN processo.dbo.Pro_Marcador pm ON pm.IdMarcador = pmp.IdMarcador 
	INNER JOIN processo.dbo.Relator r ON r.codigo = p.codigo_relator 
WHERE setor_atual = 'CCD' 
	AND lower(pm.Descricao) = 'CÁLCULO VENCIMENTOS DO GESTOR'
"""
processos_gestores = pd.read_sql(sql_gestores, conn)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_4900\2921930658.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_gestores = pd.read_sql(sql_gestores, conn)


In [54]:
llm = AzureChatOpenAI(model_name="gpt-4o")

class PessoaMultaAcordao(BaseModel):
    nome: str = Field(description="Nome da pessoa que foi multada")
    cpf: str = Field(description="CPF da pessoa que foi multada, em formato XXXXXXXXXXX, sem pontos ou traços")
    orgao: str = Field(description="Órgão ao qual a pessoa multada pertence")
    percentual_valor: str = Field(description="Percentual ou valor da multa aplicada, se houver")
    ano_aplicacao_multa: int = Field(description="Ano em que a multa na época do ato que deu causa à multa")
    cargo_gestor: str = Field(description="Cargo do gestor multado")
    descricao: str = Field(description="Trecho do acórdão que menciona a multa")

struct_llm = llm.with_structured_output(
    schema=PessoaMultaAcordao
)

def get_pessoa_multa_acordao(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa acordãos do TCE que contem multas.
        Sua tarefa é identificar o nome da pessoa que foi multada e o valor da multa, quando for o caso de multa percentual sobre vencimentos de um gestor.
        O texto do é o seguinte:
        "{input}"
       
        Sua resposta:
        """)
        chain = prompt | struct_llm
        return chain.invoke(texto)
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None


In [55]:
for i, row in processos_gestores.iterrows():
    print(f"{i}: {row['processo']} - {row['nome']} - {row['assunto']}")
    infos_processo = get_informacoes_processo(row['processo'], conn)
    texto = '\n'.join(infos_processo[infos_processo['setor'].str.contains('secpc', case=False)].texto.to_list())
    ret = get_pessoa_multa_acordao(texto)
    if ret:
        print(f"Nome: {ret.nome}, Percentual/Valor: {ret.percentual_valor}, Ano: {ret.ano_aplicacao_multa}, Cargo: {ret.cargo_gestor}, Descrição: {ret.descricao}")
        processos_gestores.loc[i, 'nome_multa'] = ret.nome
        processos_gestores.loc[i, 'cpf'] = ret.cpf
        processos_gestores.loc[i, 'orgao'] = ret.orgao 
        processos_gestores.loc[i, 'percentual_valor'] = ret.percentual_valor
        processos_gestores.loc[i, 'ano_aplicacao_multa'] = ret.ano_aplicacao_multa
        processos_gestores.loc[i, 'cargo_gestor'] = ret.cargo_gestor
        processos_gestores.loc[i, 'descricao_multa'] = ret.descricao
    else:
        print("Nenhuma multa encontrada.")
        processos_gestores.loc[i, 'nome_multa'] = None
        processos_gestores.loc[i, 'cpf'] = None
        processos_gestores.loc[i, 'orgao'] = None
        processos_gestores.loc[i, 'percentual_valor'] = None
        processos_gestores.loc[i, 'ano_aplicacao_multa'] = None
        processos_gestores.loc[i, 'cargo_gestor'] = None
        processos_gestores.loc[i, 'descricao_multa'] = None

    

0: 006552/2019 - ANTONIO GILBERTO DE OLIVEIRA JALES - APURAÇÃO DE RESPONSABILIDADE - PODER EXECUTIVO MUNICIPAL - 2015 (INFRAÇÕES A LRF/RESOLUÇÃO Nº. 004/2013 - TC).                                                                                                                                                 


c:\Users\05911205424\Documents\Dev\ccd\scripts\utils_ccd.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


Nome: Luiz Benes Leocádio de Araújo, Percentual/Valor: 5% dos vencimentos anuais e valores de R$10.500,00 (detalhado)., Ano: 2016, Cargo: Gestor Municipal, Descrição: 1. Aplicação de multa equivalente a 5% do vencimento anual por atraso de publicação do RGF; 2. Aplicação de multa de R$10.500,00 distribuída entre atrasos na publicação de RREO e outros fatores destacados.
1: 006412/2019 - PAULO ROBERTO CHAVES ALVES - APURAÇÃO DE RESPONSABILIDADE - PODER EXECUTIVO MUNICIPAL - 2015 (INFRAÇÕES A LRF/RESOLUÇÃO Nº. 004/2013 - TC).                                                                                                                                                 


c:\Users\05911205424\Documents\Dev\ccd\scripts\utils_ccd.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


Nome: , Percentual/Valor: , Ano: 0, Cargo: , Descrição: 
2: 004820/2020 - PAULO ROBERTO CHAVES ALVES - APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO DE 2016 (INFRAÇÕES A LRF)                                                                                                                                                                                       


...

Nome: ANTÔNIO CARLOS PEIXOTO NUNES, Percentual/Valor: 30% dos vencimentos anuais do Responsável no exercício 2013, Ano: 2013, Cargo: Gestor Público, Descrição: Aplicação de multa por omissão do dever de publicar os Relatórios de Gestão Fiscal do 1º e 2º semestres do exercício 2013, conforme artigo 5º, inciso I e §1º, Lei nº 10.028/2000.
22: 000263/2019 - FRANCISCO POTIGUAR CAVALCANTI JÚNIOR - APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PROCESSO N.º 6568                                                                                                                                                                                                   


c:\Users\05911205424\Documents\Dev\ccd\scripts\utils_ccd.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


Nome: José Júlio Fernandes Neto, Percentual/Valor: 30% sobre vencimentos anuais do gestor, Ano: 2014, Cargo: Gestor responsável, Descrição: multa equivalente a 30% sobre os vencimentos anuais do gestor responsável, em face da ausência de publicação do Relatório de Gestão Fiscal – RGF, do 1º semestre de 2014
23: 000548/2019 - FRANCISCO POTIGUAR CAVALCANTI JÚNIOR - APURAÇÃO DE RESPONSABILIDADE, REFERENTE A GESTÃO FISCAL DE 2014                                                                                                                                                                                                


c:\Users\05911205424\Documents\Dev\ccd\scripts\utils_ccd.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


Nome: Francisco Assis de Morais Araújo, Percentual/Valor: 30%, Ano: 2014, Cargo: Não especificado diretamente, Descrição: Julgado irregular, gestor aplicado multa de 30% sobre vencimentos anuais conforme art. 5º, §1º da Lei nº 10.028/2000 e artigo 29, inciso V, da Resolução TCE 004/2013


In [62]:
sql_vencimentos = """
SELECT
    qfr.ano,
    qfr.mes,
    s.cpf COLLATE SQL_Latin1_General_CP1_CI_AS as cpf,
   	s.nome COLLATE SQL_Latin1_General_CP1_CI_AS as nome_servidor,
    icc.DescItemContraCheque as descricao_rubrica,
    cci.valor as valor_rubrica,
	o.idOrgao as id_orgao,
    rtrim(o.codigo) as codigo_orgao,
    o.nome COLLATE SQL_Latin1_General_CP1_CI_AS as nome_orgao,
    NULL as codigo_orgao_superior,
    NULL as tipo_orgao_natureza,
    
   
    s.matricula COLLATE SQL_Latin1_General_CP1_CI_AS,
    
    s.datanascimento,
    convert(char(10), s.[dataadmissao], 126) as data_admissao,
    convert(char(10), s.[datadesligamento], 126)  as data_desligamento,
    s.cargo COLLATE SQL_Latin1_General_CP1_CI_AS as cargo,
    s.lotacao COLLATE SQL_Latin1_General_CP1_CI_AS as lotacao,
    s.pne COLLATE SQL_Latin1_General_CP1_CI_AS,
    s.dedicacaoexclusiva COLLATE SQL_Latin1_General_CP1_CI_AS as de,
    s.cargahoraria as ch,
    upper(fi.[descformaingresso]) COLLATE SQL_Latin1_General_CP1_CI_AS as forma_ingresso,
    upper(rj.[descregimejuridico]) COLLATE SQL_Latin1_General_CP1_CI_AS as regime_juridico,
    upper(v.[descricaovinculo]) COLLATE SQL_Latin1_General_CP1_CI_AS as vinculo,
    upper(sf.[descsituacaofuncional]) COLLATE SQL_Latin1_General_CP1_CI_AS as situacao_funcional,
    

    --fp.idfpfolhapagamento as id_folha,
    upper(tfp.desctipofolha) as tipo_folha,
    cc.idfpcontracheque as id_contracheque,
    cast(s.totaldesconto as numeric(32,2)) as total_descontos,
    cast(s.totalvantagem as numeric(32,2)) as total_vantagens,
    cc.datainclusao AS data_inclusao,
	0 as retificacao


FROM 
	-- remessa
    [processo].[dbo].siaidp_arquivo as a
	inner join [processo].[dbo].orgaos as o on a.codigoorgao = o.codigo

	-- folha de pagamento
	inner join [processo].[dbo].siaidp_fpremessa as rfp on a.idarquivo = rfp.idarquivo
	inner join [processo].[dbo].siaidp_fpfolhapagamento as fp on rfp.idfpremessa = fp.idfpremessa
	inner join [processo].[dbo].[siaidp_fptipofolha] as tfp on fp.idfptipofolha = tfp.idfptipofolha
	inner join [processo].[dbo].siaidp_fpcontracheque as cc on fp.idfpfolhapagamento = cc.idfpfolhapagamento
	inner join [processo].[dbo].siaidp_fpcontrachequeitem as cci on cci.idfpcontracheque = cc.idfpcontracheque
	inner join [processo].[dbo].SiaiDp_FpItemContraCheque icc on cci.idfpitemcontracheque = icc.idfpitemcontracheque
	
	-- servidor
	inner join [processo].[dbo].siaidp_qfremessa as qfr on a.idarquivo = qfr.idarquivo
	inner join [processo].[dbo].siaidp_qfservidor as s on s.idqfremessa = qfr.idqfremessa and s.cpf = cc.cpf
	left join [processo].[dbo].[siaidp_qfformaingresso] as fi on s.[idqfformaingresso] = fi.[idqfformaingresso]
	left join [processo].[dbo].[siaidp_qfregimejuridico] as rj on s.[idqfregimejuridico] = rj.[idqfregimejuridico]
    left join [processo].[dbo].[siaidp_qfsituacaofuncional] as sf on s.[idqfsituacaofuncional] = sf.[idqfsituacaofuncional]
	left join [processo].[dbo].[siaidp_qfvinculo] as v on s.[idqfvinculo] = v.[idqfvinculo]

WHERE 
    a.inativo is null
    AND s.CPF = '{}' and qfr.ano = {} 
"""

In [63]:
row = processos_gestores.iloc[0]
sql = sql_vencimentos.format(row.cpf, row.ano_aplicacao_multa)
df_row = pd.read_sql(sql, conn)


C:\Users\05911205424\AppData\Local\Temp\ipykernel_4900\1103932714.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_row = pd.read_sql(sql, conn)


In [ ]:
df_row['descricao_rubrica']

,ano,mes,cpf,nome_servidor,descricao_rubrica,valor_rubrica,id_orgao,codigo_orgao,nome_orgao,codigo_orgao_superior,...,forma_ingresso,regime_juridico,vinculo,situacao_funcional,tipo_folha,id_contracheque,total_descontos,total_vantagens,data_inclusao,retificacao
0,2016,1,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Vencimento Básico ...,17025.66,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12297122,9810.72,21120.02,2016-03-14 09:45:30.827,0
1,2016,1,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Auxílios e benefícios ...,1200.00,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12297122,9810.72,21120.02,2016-03-14 09:45:30.827,0
2,2016,1,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Anuênio ...,2894.36,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12297122,9810.72,21120.02,2016-03-14 09:45:30.827,0
3,2016,1,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Previdência (Regime Próprio) ...,2191.20,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12297122,9810.72,21120.02,2016-03-14 09:45:30.827,0
4,2016,1,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Imposto de Renda Retido na Fonte (IRRF) ...,4006.07,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,12297122,9810.72,21120.02,2016-03-14 09:45:30.827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2016,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Auxílios e benefícios ...,1500.00,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,15673437,10363.26,21590.27,2017-03-21 15:05:47.553,0
68,2016,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Anuênio ...,3064.61,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,15673437,10363.26,21590.27,2017-03-21 15:05:47.553,0
69,2016,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Previdência (Regime Próprio) ...,2209.92,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,15673437,10363.26,21590.27,2017-03-21 15:05:47.553,0
70,2016,12,40665429487,LUIZ BENES LEOCADIO DE ARAUJO ...,Imposto de Renda Retido na Fonte (IRRF) ...,4047.74,333,PL,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,None,...,CONCURSO PÚBLICO ...,ESTATUTÁRIO - ESTADUAL ...,CARGO EFETIVO ...,ATIVO ...,NORMAL ...,15673437,10363.26,21590.27,2017-03-21 15:05:47.553,0


In [56]:
processos_gestores

,processo,codigo_tipo_processo,assunto,nome,marcador,id_processo,nome_multa,percentual_valor,ano_aplicacao_multa,cargo_gestor,descricao_multa,cpf,orgao
0,006552/2019,APR,APURAÇÃO DE RESPONSABILIDADE - PODER EXECUTIVO...,ANTONIO GILBERTO DE OLIVEIRA JALES,CÁLCULO VENCIMENTOS DO GESTOR,512216,Luiz Benes Leocádio de Araújo,5% dos vencimentos anuais e valores de R$10.50...,2016.0,Gestor Municipal,1. Aplicação de multa equivalente a 5% do venc...,40665429487,Prefeitura Municipal de Lajes/RN
1,006412/2019,APR,APURAÇÃO DE RESPONSABILIDADE - PODER EXECUTIVO...,PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,511916,,,0.0,,,,
2,004820/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,539327,Não há informações sobre a pessoa multada no t...,,0.0,,O campo de entrada de texto não contém informa...,,
3,004675/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,538546,,,0.0,,,,
4,004677/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,538554,,,0.0,,,,
5,004332/2019,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO EXE...",GEORGE MONTENEGRO SOARES,CÁLCULO VENCIMENTOS DO GESTOR,505826,,,0.0,,O texto fornecido não possui conteúdo para aná...,,
6,004330/2019,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO EXE...",ANTONIO ED SOUZA SANTANA,CÁLCULO VENCIMENTOS DO GESTOR,505824,Paulo Marcio de Lima Fernandes,30% dos vencimentos anuais do presidente da Câ...,2015.0,Presidente da Câmara Municipal de Umarizal/RN,Imputação de multa equivalente a 30% dos venci...,03619913471,Câmara Municipal de Umarizal/RN
7,004230/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,536990,,,0.0,,,,
8,004233/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",ANTONIO ED SOUZA SANTANA,CÁLCULO VENCIMENTOS DO GESTOR,536995,,,0.0,,,,
9,003788/2020,APR,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO ANO...",PAULO ROBERTO CHAVES ALVES,CÁLCULO VENCIMENTOS DO GESTOR,535458,,,0.0,,"""""",,
